In [28]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from tracker import*
import time

In [29]:
model=YOLO('yolov8s.pt')

In [30]:
def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)

In [31]:
cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

In [32]:
cap=cv2.VideoCapture('veh2.mp4')

In [33]:
my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n") 
#print(class_list)

In [34]:
count=0
tracker=Tracker()
vh_down = {}
counter = []
vh_up = {}
counter1 = []
vh_dtime = {}
vh_utime = {}

In [35]:
cy1=322
cy2=368
offset=6

In [36]:
while True:    
    ret,frame = cap.read()
    if not ret:
        break
    count += 1
    if count % 3 != 0:
        continue
    frame=cv2.resize(frame,(1020,500))
   

    results=model.predict(frame)
 #   print(results)
    a=results[0].boxes.data
    px=pd.DataFrame(a).astype("float")
#    print(px)
    list=[]
             
    for index,row in px.iterrows():
#        print(row)
 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if 'car' in c:
            list.append([x1,y1,x2,y2])
    bbox_id=tracker.update(list)
    for bbox in bbox_id:
        x3,y3,x4,y4,id=bbox
        cx=int(x3+x4)//2
        cy=int(y3+y4)//2
        cv2.rectangle(frame,(x3,y3),(x4,y4),(0,0,255),1)
        if cy1<(cy+offset) and cy1>(cy-offset):
                vh_down[id] = cy
                vh_dtime[id] = time.time()
        if id in vh_down:
            if cy2<(cy+offset) and cy2>(cy-offset):
                elapsed_time = time.time()-vh_dtime[id]
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
                if counter.count(id)==0:
                    counter.append(id)
                distance = 10
                speed_ms = distance/elapsed_time
                speed_km = speed_ms * 3.6
                cv2.putText(frame,str(int(speed_km))+'Km/h',(x4,y4),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
   
################### going down ######################### 

        if cy2<(cy+offset) and cy2>(cy-offset):
                vh_up[id] = cy
                vh_utime[id] = time.time()
        if id in vh_up:
            if cy1<(cy+offset) and cy1>(cy-offset):
                elapsed_time2 = time.time()-vh_utime[id]
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
                if counter1.count(id)==0:
                    counter1.append(id)
                distance2 = 10
                speed_ms2 = distance2/elapsed_time2
                speed_km2 = speed_ms2 * 3.6
                cv2.putText(frame,str(int(speed_km2))+'Km/h',(x4,y4),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)


    cv2.line(frame,(267,cy1),(829,cy1),(255,255,255),1)
    cv2.putText(frame,('1line'),(274,318),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    
    cv2.line(frame,(167,cy2),(932,cy2),(255,255,255),1)
    cv2.putText(frame,('2line'),(181,363),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    
    d = (len(counter))
    cv2.putText(frame,('GoingDown : ')+str(d),(60,40),cv2.FONT_HERSHEY_COMPLEX,0.7,(255,0,0),2)

    d2 = (len(counter1))
    cv2.putText(frame,('GoingUp : ')+str(d2),(60,130),cv2.FONT_HERSHEY_COMPLEX,0.7,(255,0,0),2)
    
    cv2.imshow("RGB", frame)
    if cv2.waitKey(1)&0xFF==27:
        break
         
cap.release()
cv2.destroyAllWindows()

Ultralytics YOLOv8.0.20  Python-3.9.13 torch-2.1.0+cpu CPU
YOLOv8s summary (fused): 168 layers, 11156544 parameters, 0 gradients, 28.6 GFLOPs


[909, 11]
